<a href="https://colab.research.google.com/github/kapamawi/AI/blob/main/2_5_4___multihop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
!pip install -qU langchain langchain-openai langchain-community pydantic chromadb

Ten kod instaluje kilka bibliotek Pythona używając menedżera pakietów pip. Flaga -q włącza tryb cichy (quiet), który ukrywa szczegóły procesu instalacji, a -U wymusza aktualizację do najnowszych wersji.

Instalowane są następujące biblioteki:

langchain - framework do budowania aplikacji wykorzystujących modele językowe. Pozwala łączyć różne modele i narzędzia w złożone łańcuchy przetwarzania.

langchain-openai - wtyczka do LangChain umożliwiająca integrację z modelami OpenAI jak GPT-3 i GPT-4.

langchain-community - zbiór komponentów stworzonych przez społeczność LangChain, zawierający dodatkowe narzędzia i integracje.

pydantic - biblioteka do walidacji danych i zarządzania ustawieniami w Pythonie. Jest używana przez LangChain do definiowania schematów danych.

chromadb - baza danych wektorowych, która pozwala na przechowywanie i wyszukiwanie danych tekstowych na podstawie ich znaczenia semantycznego. Jest często używana z LangChain do tworzenia systemów wyszukiwania i rekomendacji.

Te biblioteki razem tworzą środowisko do budowania zaawansowanych aplikacji wykorzystujących sztuczną inteligencję, szczególnie w zakresie przetwarzania języka naturalnego i wyszukiwania semantycznego.

In [ ]:
import os
import numpy as np
from pydantic import BaseModel, Field
from langchain import hub
from langchain.agents import (
    AgentExecutor,   Tool,  create_openai_tools_agent,  create_react_agent,
    tool
)
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import LocalFileStore
from langchain.storage._lc_store import create_kv_docstore
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool
from langchain.vectorstores import Chroma
from google.colab import userdata
from pprint import pp as ppr

Ten kod importuje szereg bibliotek i narzędzi potrzebnych do stworzenia systemu przetwarzania języka naturalnego.

Importy można podzielić na kilka logicznych grup:

Z biblioteki standardowej importowany jest moduł os do operacji systemowych.

Z numpy (jako np) importowane są funkcje do obliczeń numerycznych.

Z pydantic pobierane są BaseModel i Field do definiowania modeli danych z walidacją.

Z ekosystemu LangChain importowane są komponenty do:

- Tworzenia agentów AI (AgentExecutor, Tool, create_openai_tools_agent, create_react_agent, tool) - są to elementy pozwalające na budowę systemów podejmujących autonomiczne decyzje

- Łańcuchów przetwarzania (RetrievalQA) - do tworzenia systemów pytanie-odpowiedź

- Integracji z OpenAI (ChatOpenAI, OpenAIEmbeddings) - umożliwiają komunikację z modelami GPT i tworzenie reprezentacji tekstów

- Ładowania dokumentów (TextLoader) - do wczytywania plików tekstowych

- Zarządzania promptami (ChatPromptTemplate, MessagesPlaceholder) - służą do formatowania zapytań do modeli

- Wyszukiwania dokumentów (ParentDocumentRetriever) - pozwala na inteligentne przeszukiwanie bazy dokumentów

- Przechowywania danych (LocalFileStore, create_kv_docstore) - do lokalnego magazynowania informacji

- Dzielenia tekstu (RecursiveCharacterTextSplitter) - do podziału długich tekstów na mniejsze fragmenty

- Narzędzi wyszukiwania (create_retriever_tool) - do tworzenia narzędzi przeszukujących dokumenty

- Bazy wektorowej (Chroma) - do przechowywania i wyszukiwania tekstów na podstawie ich znaczenia

Dodatkowo importowane są narzędzia pomocnicze:
- userdata z google.colab do obsługi danych użytkownika w środowisku Google Colab
- ppr (alias dla pprint) do ładnego wyświetlania złożonych struktur danych
.

In [ ]:
class CFG:
  model1 = "gpt-4o-mini"
  model2 = "text-embedding-3-large"
  temperature = 0.1
  chunksize = 1024

In [ ]:

os.environ['OPENAI_API_KEY'] = userdata.get('openaivision')

Ta linia kodu konfiguruje klucz API do usług OpenAI w zmiennych środowiskowych. Dzieje się to w dwóch krokach:

userdata.get('openaivision') pobiera zapisany wcześniej klucz API ze specjalnego miejsca w Google Colab, gdzie przechowywane są poufne dane użytkownika. Jest to bezpieczniejszy sposób przechowywania kluczy API niż wpisywanie ich bezpośrednio w kodzie, ponieważ chroni je przed przypadkowym udostępnieniem.

Następnie ten klucz jest przypisywany do zmiennej środowiskowej o nazwie 'OPENAI_API_KEY'. Zmienne środowiskowe to specjalne zmienne systemowe dostępne dla wszystkich programów uruchomionych w danym środowisku. Biblioteki OpenAI automatycznie szukają klucza API właśnie w tej zmiennej środowiskowej.


In [ ]:

llm = ChatOpenAI(model= CFG.model1)

embeddings = OpenAIEmbeddings(model = CFG.model2)


<ipython-input-6-2be88f6e64f2>:3: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(model = CFG.model2)


Te dwie linie kodu tworzą fundamentalne komponenty do pracy z API OpenAI.

`llm = ChatOpenAI(model= CFG.model1)` tworzy obiekt modelu językowego do prowadzenia rozmów, używając modelu określonego w konfiguracji jako CFG.model1. Ten obiekt będzie używany do generowania odpowiedzi w formacie konwersacji.

`embeddings = OpenAIEmbeddings(model = CFG.model2)` tworzy obiekt do generowania embedingów tekstu, używając modelu zdefiniowanego w zmiennej CFG.model2. Embeddingi to wektorowe (liczbowe) reprezentacje tekstu, które zachowują jego znaczenie - teksty o podobnym znaczeniu mają podobne reprezentacje wektorowe. Jest to kluczowe dla wyszukiwania semantycznego i porównywania tekstów pod względem znaczenia, nie tylko dopasowania słów.

Oba obiekty używają wcześniej skonfigurowanego klucza API OpenAI do komunikacji z serwerami OpenAI.

# Data

In [ ]:
loader = TextLoader("/content/globalcorp_hr_policy.txt")
documents = loader.load()

Te dwie linie kodu służą do wczytania dokumentu tekstowego:

`loader = TextLoader("/content/globalcorp_hr_policy.txt")` tworzy obiekt ładujący, który będzie czytał plik tekstowy zawierający zasady HR firmy GlobalCorp. Plik znajduje się w ścieżce '/content/globalcorp_hr_policy.txt'.

`documents = loader.load()` wykonuje właściwe wczytanie dokumentu. Metoda load() przetwarza plik tekstowy i zwraca jego zawartość w formie, która może być dalej przetwarzana przez system. Zawartość jest zapisywana do zmiennej documents.

Jest to pierwszy krok w procesie przetwarzania dokumentów - po wczytaniu tekst może być podzielony na mniejsze części, przekształcony w embeddingi i zapisany w bazie wektorowej do późniejszego wyszukiwania.

In [ ]:
persist_directory = "local_vectorstore"
local_store = "local_docstore"
collection_name = "hrpolicy"
PROJECT_ROOT = "/content/"


# Retriever

In [ ]:

parent_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
child_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)

Te linie tworzą dwa różne narzędzia do dzielenia tekstu:

`parent_splitter` dzieli tekst na większe fragmenty po 1000 znaków z nakładką 20 znaków między fragmentami. Jest to "rodzic" w hierarchii podziału tekstu.

`child_splitter` tworzy mniejsze fragmenty po 200 znaków, również z nakładką 20 znaków. Jest to "dziecko" w hierarchii podziału tekstu.

RecursiveCharacterTextSplitter działa rekurencyjnie, co oznacza że stara się inteligentnie dzielić tekst, zachowując spójność zdań i akapitów. Nakładka (overlap) między fragmentami pomaga zachować kontekst na granicach podziału - te same 20 znaków pojawia się na końcu jednego fragmentu i na początku następnego.

Taki dwupoziomowy podział pozwala na efektywne wyszukiwanie - najpierw w większych fragmentach, a potem w ich mniejszych częściach, co zwiększa dokładność odpowiedzi systemu.

In [ ]:

vectorstore = Chroma(
    persist_directory=os.path.join(PROJECT_ROOT, "data", persist_directory),
    collection_name=collection_name,
    embedding_function=embeddings,
)

Ta linia kodu tworzy bazę danych wektorowych Chroma do przechowywania i wyszukiwania dokumentów. Rozłóżmy jej komponenty:

`persist_directory` określa ścieżkę, gdzie będą zapisywane dane. Jest ona tworzona przez połączenie PROJECT_ROOT (główny katalog projektu), podkatalogu "data" i wartości zmiennej persist_directory.

`collection_name` nadaje nazwę kolekcji dokumentów w bazie. Kolekcje pozwalają na organizację różnych zestawów dokumentów.

`embedding_function=embeddings` wskazuje, że do tworzenia wektorowych reprezentacji tekstów ma być użyty wcześniej skonfigurowany obiekt embeddings (który używa modelu OpenAI).

Chroma to baza danych zoptymalizowana do przechowywania embedingów i szybkiego wyszukiwania podobnych dokumentów. Działa jak zaawansowana wyszukiwarka, która rozumie znaczenie tekstu, nie tylko słowa kluczowe.

In [ ]:

local_store = LocalFileStore(os.path.join(PROJECT_ROOT, "data", local_store))
store = create_kv_docstore(local_store)
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

<ipython-input-9-75800798405b>:5: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


Te linie kodu tworzą system do przechowywania i wyszukiwania dokumentów.

`local_store = LocalFileStore(...)` tworzy lokalny magazyn plików w katalogu projektu. To miejsce do przechowywania oryginalnych dokumentów.

`store = create_kv_docstore(local_store)` tworzy magazyn typu klucz-wartość na podstawie lokalnego magazynu plików. Pozwala na szybkie odnajdywanie dokumentów po ich identyfikatorach.

Ostatni fragment tworzy zaawansowany system wyszukiwania dokumentów:
```python
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)
```
- `vectorstore` to wcześniej utworzona baza Chroma do przechowywania wektorów
- `docstore` to magazyn oryginalnych dokumentów
- `child_splitter` i `parent_splitter` to wcześniej zdefiniowane narzędzia do dzielenia tekstu

Ten system działa dwupoziomowo:
1. Najpierw wyszukuje odpowiednie większe fragmenty tekstu (używając parent_splitter)
2. Następnie w znalezionych fragmentach szuka dokładniejszych dopasowań (używając child_splitter)

Taka struktura zapewnia zarówno dokładność jak i szybkość wyszukiwania.

# Functions

In [ ]:
def value_to_float(x):
    if int(x):
        return int(x)
    if type(x) == float or type(x) == int:
        return x
    x = x.upper()
    if "MILLION" in x:
        if len(x) > 1:
            return float(x.replace("MILLION", "")) * 1000000
        return 1000000.0
    if "BILLION" in x:
        return float(x.replace("BILLION", "")) * 1000000000

    return 0.0


Ta funkcja `value_to_float` konwertuje różne formaty zapisu liczb na wartości zmiennoprzecinkowe. Przeanalizujmy jak działa:

1. Jeśli wartość da się przekonwertować na liczbę całkowitą (`int(x)`), zwraca tę liczbę.

2. Jeśli wartość jest już typu float lub int, zwraca ją bez zmian.

3. Dla tekstu:
   - Zamienia tekst na wielkie litery dla spójności
   - Jeśli zawiera słowo "MILLION":
     - Usuwa słowo "MILLION" i mnoży pozostałą liczbę przez 1,000,000
     - Jeśli nie ma liczby przed "MILLION", zwraca 1,000,000
   - Jeśli zawiera słowo "BILLION":
     - Usuwa słowo "BILLION" i mnoży pozostałą liczbę przez 1,000,000,000

4. Jeśli żaden z powyższych przypadków nie pasuje, zwraca 0.0

Funkcja może przetworzyć na przykład:
- "5 MILLION" na 5,000,000.0
- "2.5 BILLION" na 2,500,000,000.0
- "100" na 100
- "MILLION" na 1,000,000.0

Jest przydatna do przetwarzania danych finansowych lub statystycznych, gdzie liczby często są zapisywane w różnych formatach tekstowych.

In [ ]:


def convert_currency_to_usd(amount: str) -> int:
    "Converts currency into USD"

    if "¥" in amount:
        exclude_symbol = amount.replace("¥", "")
        amount_in_numbers = value_to_float(exclude_symbol)
        return amount_in_numbers / 147.72
    if "$" in amount:
        exclude_symbol = amount.replace("$", "")
        return value_to_float(exclude_symbol)
    if "JPY" in amount:
        exclude_symbol = amount.replace("JPY", "")
        return int(exclude_symbol) / 147.72
    if "USD" in amount:
        return amount


Ta funkcja `convert_currency_to_usd` konwertuje kwoty z różnych walut na dolary amerykańskie (USD).

Funkcja obsługuje następujące formaty:
- Kwoty w jenach japońskich oznaczone symbolem ¥ lub kodem JPY
- Kwoty w dolarach amerykańskich oznaczone symbolem $ lub kodem USD

Proces konwersji przebiega następująco:
1. Dla kwot w jenach (¥ lub JPY):
   - Usuwa symbol waluty
   - Konwertuje tekst na liczbę przy pomocy wcześniejszej funkcji value_to_float
   - Dzieli przez kurs wymiany 147.72 jenów za dolara

2. Dla kwot w dolarach ($):
   - Usuwa symbol dolara
   - Konwertuje wartość na liczbę

3. Dla kwot oznaczonych jako USD:
   - Zwraca wartość bez zmian

Na przykład:
- "¥1000" zostanie przeliczone na około 6.77 USD
- "$100" zostanie zamienione na 100 USD
- "1000 JPY" zostanie przeliczone na około 6.77 USD

# QnA system

In [ ]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
)

Ten kod tworzy system pytań i odpowiedzi (QA) wykorzystujący model językowy i bazę wiedzy. Rozłóżmy go na części:

`RetrievalQA.from_chain_type()` tworzy system QA, który może wyszukiwać informacje i generować na ich podstawie odpowiedzi.

Parametry przekazane do tej metody:
- `llm=llm` - określa model językowy, który będzie generował odpowiedzi
- `chain_type="stuff"` - wybiera najprostszą strategię łączenia znalezionych fragmentów tekstu - wszystkie są po prostu sklejane w jeden dokument
- `retriever=retriever` - wskazuje komponent odpowiedzialny za wyszukiwanie odpowiednich fragmentów w bazie wiedzy
- `return_source_documents=True` - sprawia, że system będzie zwracał nie tylko odpowiedź, ale też dokumenty źródłowe, z których została wygenerowana

Całość tworzy pipeline, który po otrzymaniu pytania:
1. Przeszuka bazę wiedzy
2. Znajdzie właściwe fragmenty tekstu
3. Połączy je w jeden dokument
4. Użyje modelu językowego do wygenerowania odpowiedzi
5. Zwróci odpowiedź wraz z źródłami

## Plusy

In [ ]:
response = qa({"query": "What is the allocated budget for communication initiatives?"})
ppr(response)

<ipython-input-15-f7edc09cb693>:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa({"query": "What is the allocated budget for communication initiatives?"})


{'query': 'What is the allocated budget for communication initiatives?',
 'result': 'The allocated budget for communication initiatives is $500,000 in '
           'the U.S., €250,000 in Germany, and ¥25 million in Japan.',
 'source_documents': [Document(metadata={'source': '/content/globalcorp_hr_policy.txt'}, page_content='**Health and Safety:**\nWorkplace safety is a shared responsibility. Emergency procedures are clearly posted throughout our office buildings, and an annual budget of $10,000, €5,000, and ¥1 million is allocated for safety drills and equipment maintenance in the U.S., Germany, and Japan, respectively.\n\n**Communication:**\nImportant updates are conveyed through company-wide emails and team meetings. An annual budget of $500,000, €250,000, and ¥25 million is allocated for communication initiatives, including employee engagement events in the U.S., Germany, and Japan, respectively.\n\nThis policy undergoes an annual review to ensure relevance and compliance. Welcome 

In [ ]:
qa({"query": "How many maternity leaves are offered?"})

{'query': 'How many maternity leaves are offered?',
 'result': "I don't know.",
 'source_documents': [Document(metadata={'source': '/content/globalcorp_hr_policy.txt'}, page_content='2.4\t<Company> may, on occasions, decide to restrict advertisement to internal candidates only. Furthermore junior posts will always be advertised internally in the first instance, to provide continuous development of existing members of staff unless the Management team agrees that this is not appropriate due to the specialist skills required for the post involved. \n\n2.5\tVacancies, which are restricted to internal candidates only will be clearly indicated on the advertisement. All internal candidates will be selected for interview on the same criteria as external candidates. \n\n2.6\tEmployees on maternity leave will receive all advertisements for posts advertised in the Company during their period of maternity leave.\n3.\tApplication Form\n\n3.1\tCandidates for all posts will, except on some occasions 

In [ ]:
qa({"query": "What is the probationary period?"})

{'query': 'What is the probationary period?',
 'result': "The probationary period is a duration of six calendar months during which all appointments into the Company are made. After three months, a review meeting takes place between the post holder and their line manager to discuss progress. At the end of the probationary period, if a satisfactory report is provided by the appropriate head of section or line manager, the employee will be notified in writing that they have successfully completed their probationary period. The probationary period can also be extended by a further three months if deemed appropriate by the individual's line manager.",
 'source_documents': [Document(metadata={'source': '/content/globalcorp_hr_policy.txt'}, page_content='•\ttwo satisfactory references;\n•\tconfirmation of the right to work in this country (if appropriate)\n•\tCriminal Records Disclosure (if appropriate).\n6.\tProbationary Period\n\t\n6.1\tAll appointments into the Company will be made subjec

In [ ]:
qa({"query": "Is the work hours in Germany different than United States?"})

{'query': 'Is the work hours in Germany different than United States?',
 'result': 'Yes, the work hours in Germany are different than in the United States. In Germany, the standard workweek is 38 hours (Monday to Friday, 8 AM to 5 PM), while in the United States, employees adhere to a standard 40-hour workweek (Monday to Friday, 9 AM to 5 PM).',
 'source_documents': [Document(metadata={'source': '/content/globalcorp_hr_policy.txt'}, page_content='**GlobalCorp Human Resources Policy**\n\nWelcome to GlobalCorp, where our Human Resources Policy is designed to provide a comprehensive framework for employees across our offices in the United States, Germany, and Japan. We operate under an at-will employment relationship, and any contractual agreements should be documented in writing.\n\nAt the core of our culture is a commitment to professionalism and ethical conduct. Clear and respectful communication is highly valued, and a business casual dress code is encouraged.\n\n**Work Hours:**\nEmpl

In [ ]:
qa({"query": "What is the probationary period for new employees in USA?"})

{'query': 'What is the probationary period for new employees in USA?',
 'result': "I don't know.",
 'source_documents': [Document(metadata={'source': '/content/globalcorp_hr_policy.txt'}, page_content='•\ttwo satisfactory references;\n•\tconfirmation of the right to work in this country (if appropriate)\n•\tCriminal Records Disclosure (if appropriate).\n6.\tProbationary Period\n\t\n6.1\tAll appointments into the Company will be made subject to a probationary period of six calendar months. After three months a review meeting will take place between the post holder and their line manager to discuss progress. At the end of the probationary period, and subject to a satisfactory report by the appropriate head of section or line manager, employees will be notified in writing that they have successfully completed their probationary period. The probationary period can be extended by a further 3 months should the individuals line manager consider this appropriate.\n7.\tRecruitment Monitoring')]

In [ ]:
qa( {  "query": "What is the difference in the number of work hours in Germany vs. United States?" } )

{'query': 'What is the difference in the number of work hours in Germany vs. United States?',
 'result': 'The standard workweek in Germany is 38 hours, while in the United States it is 40 hours. Therefore, the difference in the number of work hours is 2 hours, with the United States having a longer workweek.',
 'source_documents': [Document(metadata={'source': '/content/globalcorp_hr_policy.txt'}, page_content='**GlobalCorp Human Resources Policy**\n\nWelcome to GlobalCorp, where our Human Resources Policy is designed to provide a comprehensive framework for employees across our offices in the United States, Germany, and Japan. We operate under an at-will employment relationship, and any contractual agreements should be documented in writing.\n\nAt the core of our culture is a commitment to professionalism and ethical conduct. Clear and respectful communication is highly valued, and a business casual dress code is encouraged.\n\n**Work Hours:**\nEmployees in the United States adhere to

In [ ]:
qa({"query": "Can I reimburse travel expenses?"})

{'query': 'Can I reimburse travel expenses?',
 'result': 'Yes, candidates attending an interview will be reimbursed any reasonable expenses incurred, which typically includes traveling expenses at the appropriate second-class rail fare. However, if a candidate withdraws or refuses an offer of appointment for reasons considered inadequate by the Company, they will not be reimbursed for their expenses.',
 'source_documents': [Document(metadata={'source': '/content/globalcorp_hr_policy.txt'}, page_content='4.6\t\t\tIt should be remembered that an interview is a two way process, and candidates will be given every opportunity to view the offices where they will work and ask questions about the Company, to ensure that they have a full understanding of the post for which they are applying and the way the Company operates.\n\n4.7\t\t\tIn addition to interviews, a range of other selection techniques may be used. In such circumstances reasonable notice and relevant information will be given to e

In [ ]:
qa( { "query": "I started with the company on 1 December 2023.\
    Is my probationary period over if the date today is 26 Jan 2024?"  } )

{'query': 'I started with the company on 1 December 2023.    Is my probationary period over if the date today is 26 Jan 2024?',
 'result': 'No, your probationary period is not over yet. Since you started on 1 December 2023 and the probationary period lasts for six calendar months, it will end on 1 June 2024.',
 'source_documents': [Document(metadata={'source': '/content/globalcorp_hr_policy.txt'}, page_content='•\ttwo satisfactory references;\n•\tconfirmation of the right to work in this country (if appropriate)\n•\tCriminal Records Disclosure (if appropriate).\n6.\tProbationary Period\n\t\n6.1\tAll appointments into the Company will be made subject to a probationary period of six calendar months. After three months a review meeting will take place between the post holder and their line manager to discuss progress. At the end of the probationary period, and subject to a satisfactory report by the appropriate head of section or line manager, employees will be notified in writing that th

## Minusy

In [ ]:
# złe przeliczenie
qa( {  "query": "What is the percentage difference in the annual budget for Japan and US?" })

{'query': 'What is the percentage difference in the annual budget for Japan and US?',
 'result': "I don't know.",
 'source_documents': [Document(metadata={'source': '/content/globalcorp_hr_policy.txt'}, page_content='**Grievance and Disciplinary Procedures:**\nOur grievance and disciplinary procedures are outlined on the company intranet. Termination conditions may include gross misconduct or repeated policy violations. In such cases, a disciplinary process will be followed, including a three-strike system, before termination. Employees leaving GlobalCorp should follow the exit process detailed in the employee handbook.\n\n**Confidentiality and Data Security:**\nEmployees are expected to maintain confidentiality, and strict data security measures are in place. An annual budget of $1 million is allocated for cybersecurity training and awareness programs in the U.S., while budgets for Germany and Japan are €500,000 and ¥50 million, respectively.'),
  Document(metadata={'source': '/conten

In [ ]:
qa(  {  "query": "What is the percentage difference in the annual budget for Japan and US if 1 USD = 147.72 JPY?" })

{'query': 'What is the percentage difference in the annual budget for Japan and US if 1 USD = 147.72 JPY?',
 'result': 'To find the percentage difference in the annual budget for Japan and the U.S., we first need to convert the budgets into the same currency. \n\nThe annual budget for employee benefits is:\n- U.S.: $1,000,000\n- Japan: ¥50,000,000\n\nFirst, we convert the budget for Japan into USD:\n\\[\n\\text{Japan Budget in USD} = \\frac{¥50,000,000}{147.72} \\approx 338,095.50\n\\]\n\nNow, we have:\n- U.S. Budget: $1,000,000\n- Japan Budget: $338,095.50\n\nNext, we calculate the percentage difference using the formula:\n\\[\n\\text{Percentage Difference} = \\frac{\\text{U.S. Budget} - \\text{Japan Budget}}{\\text{U.S. Budget}} \\times 100\n\\]\n\\[\n\\text{Percentage Difference} = \\frac{1,000,000 - 338,095.50}{1,000,000} \\times 100 \\approx 66.19\\%\n\\]\n\nThus, the percentage difference in the annual budget for Japan and the U.S. is approximately **66.19%**.',
 'source_document

In [ ]:
qa({"query": "Which country has the highest budget?"})

{'query': 'Which country has the highest budget?',
 'result': 'The United States has the highest budget allocated for employee benefits, with an annual budget of $1 million.',
 'source_documents': [Document(metadata={'source': '/content/globalcorp_hr_policy.txt'}, page_content='**Grievance and Disciplinary Procedures:**\nOur grievance and disciplinary procedures are outlined on the company intranet. Termination conditions may include gross misconduct or repeated policy violations. In such cases, a disciplinary process will be followed, including a three-strike system, before termination. Employees leaving GlobalCorp should follow the exit process detailed in the employee handbook.\n\n**Confidentiality and Data Security:**\nEmployees are expected to maintain confidentiality, and strict data security measures are in place. An annual budget of $1 million is allocated for cybersecurity training and awareness programs in the U.S., while budgets for Germany and Japan are €500,000 and ¥50 mil

# Agent

In [ ]:
tool_search = create_retriever_tool(
    retriever,
    "search_hr_policy",
    "Searches and returns excerpts from the HR policy.",
)

Ten kod tworzy narzędzie wyszukiwania w dokumentach HR. Rozłóżmy go na elementy:

`create_retriever_tool()` tworzy narzędzie, które można używać w systemie do automatycznego wyszukiwania informacji. Przyjmuje trzy parametry:

- `retriever` - komponent odpowiedzialny za samo wyszukiwanie w dokumentach
- `"search_hr_policy"` - nazwa narzędzia, używana do jego identyfikacji w systemie
- `"Searches and returns excerpts from the HR policy."` - opis działania narzędzia, który wyjaśnia jego funkcję

Stworzone narzędzie jest zapisywane do zmiennej `tool_search` i może być później użyte do przeszukiwania dokumentów HR i zwracania znalezionych fragmentów tekstu --> w szczególności jako część rozwiązanai typu agentowego.

In [ ]:
currency_conversion = Tool(
    name="Currency_conversion",
    func=convert_currency_to_usd,
    description="useful for converting currency into USD. Input should be an amount.",
)

Ten kod definiuje narzędzie do konwersji walut. Rozbierzmy jego składowe:

`Tool()` tworzy nowe narzędzie z trzema parametrami:

- `name="Currency_conversion"` - nadaje narzędziu nazwę, która będzie używana do jego identyfikacji w systemie
- `func=convert_currency_to_usd` - wskazuje funkcję, która będzie wykonywana przez to narzędzie (w tym przypadku funkcję konwertującą kwoty na dolary amerykańskie)
- `description="useful for converting currency into USD. Input should be an amount."` - zapewnia opis narzędzia, który wyjaśnia jego przeznaczenie i format danych wejściowych

Całość tworzy narzędzie zapisane w zmiennej `currency_conversion`, które można wykorzystać do automatycznej konwersji kwot na USD w ramach większego systemu.

In [ ]:
prompt_react = hub.pull("hwchase17/react")

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


Ten kod pobiera predefiniowany prompt o nazwie "react" ze zbioru promptów stworzonych przez użytkownika hwchase17.

`hub.pull()` to metoda, która pobiera zasoby z repozytorium (huba).

Parametr `"hwchase17/react"` wskazuje:
- `hwchase17` - nazwę użytkownika/autora
- `react` - nazwę konkretnego promptu

Pobrany prompt jest zapisywany do zmiennej `prompt_react` i może być później wykorzystany w systemie, np. do strukturyzowania interakcji z modelem językowym według wzorca ReAct (Reasoning and Acting).

In [ ]:
prompt_react

PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'}, template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}')

In [ ]:
# jedno narzędzie
tools = [tool_search]

react_agent = create_react_agent(llm, tools=tools, prompt=prompt_react)

Te dwie linie kodu tworzą agenta używającego podejścia ReAct. Rozbierzmy to:

`tools = [tool_search]` tworzy listę narzędzi zawierającą tylko jedno narzędzie - wcześniej zdefiniowane narzędzie wyszukiwania.

`create_react_agent()` tworzy agenta działającego według wzorca ReAct (Reasoning and Acting) i przyjmuje trzy parametry:
- `llm` - model językowy, który będzie używany przez agenta
- `tools=tools` - lista dostępnych narzędzi (w tym przypadku tylko narzędzie wyszukiwania)
- `prompt=prompt_react` - wcześniej pobrany prompt, który określa sposób działania agenta

Stworzony agent jest zapisywany w zmiennej `react_agent` i może analizować problemy, planować działania i wykorzystywać przypisane mu narzędzie do wykonywania zadań.

In [ ]:
react_agent_executor = AgentExecutor(
    agent=react_agent, tools=tools, verbose=True, handle_parsing_errors=True
)

Ten kod tworzy wykonawcę dla wcześniej zdefiniowanego agenta. `AgentExecutor` zarządza wykonywaniem działań agenta i przyjmuje następujące parametry:

- `agent=react_agent` - wskazuje agenta, którego działania mają być wykonywane
- `tools=tools` - przekazuje listę narzędzi dostępnych dla agenta
- `verbose=True` - włącza szczegółowe logowanie, dzięki czemu widoczne są wszystkie kroki podejmowane przez agenta
- `handle_parsing_errors=True` - włącza obsługę błędów parsowania, co pozwala na kontynuowanie działania nawet w przypadku problemów z przetwarzaniem odpowiedzi

Stworzony wykonawca jest zapisywany w zmiennej `react_agent_executor` i służy do uruchamiania agenta w kontrolowany sposób, z odpowiednią obsługą błędów i monitorowaniem jego działań.

In [ ]:
query = "As per the HR policy, Which country has the highest budget?"
react_agent_executor.invoke({"input": query})



> Entering new AgentExecutor chain...
To find the answer to the question regarding which country has the highest budget according to the HR policy, I need to search through the HR policy documents for relevant information. 

Action: search_hr_policy  
Action Input: 'Which country has the highest budget?'  
**Grievance and Disciplinary Procedures:**
Our grievance and disciplinary procedures are outlined on the company intranet. Termination conditions may include gross misconduct or repeated policy violations. In such cases, a disciplinary process will be followed, including a three-strike system, before termination. Employees leaving GlobalCorp should follow the exit process detailed in the employee handbook.

**Confidentiality and Data Security:**
Employees are expected to maintain confidentiality, and strict data security measures are in place. An annual budget of $1 million is allocated for cybersecurity training and awareness programs in the U.S., while budgets for Germany and Jap

{'input': 'As per the HR policy, Which country has the highest budget?',
 'output': 'The country with the highest budget according to the HR policy is the United States, with an annual budget of $1 million for both cybersecurity training and employee benefits.'}

In [ ]:
query = "Which of the two countries has the highest budget - Japan or Unites States?"
react_agent_executor.invoke({"input": query})



> Entering new AgentExecutor chain...
I need to compare the budgets of Japan and the United States to determine which country has the highest budget. This information is likely to be found in recent economic reports or budget summaries for both countries. I will perform a search to find the most accurate and current data on their respective budgets. 

Action: search_hr_policy
Action Input: 'Japan budget comparison United States budget 2023'
**Grievance and Disciplinary Procedures:**
Our grievance and disciplinary procedures are outlined on the company intranet. Termination conditions may include gross misconduct or repeated policy violations. In such cases, a disciplinary process will be followed, including a three-strike system, before termination. Employees leaving GlobalCorp should follow the exit process detailed in the employee handbook.

**Confidentiality and Data Security:**
Employees are expected to maintain confidentiality, and strict data security measures are in place. An 

{'input': 'Which of the two countries has the highest budget - Japan or Unites States?',
 'output': 'I was unable to find specific information comparing the national budgets of Japan and the United States for 2023. You may want to check recent government budget documents or reliable economic news sources for accurate budget figures.'}

In [ ]:
query = "How much is the budget for Japan different than United States?"
react_agent_executor.invoke({"input": query})



> Entering new AgentExecutor chain...
To find the information regarding the budget differences between Japan and the United States, I should search for relevant HR policies or budget documents that outline these details. 

Action: search_hr_policy  
Action Input: 'budget comparison Japan United States'  **Grievance and Disciplinary Procedures:**
Our grievance and disciplinary procedures are outlined on the company intranet. Termination conditions may include gross misconduct or repeated policy violations. In such cases, a disciplinary process will be followed, including a three-strike system, before termination. Employees leaving GlobalCorp should follow the exit process detailed in the employee handbook.

**Confidentiality and Data Security:**
Employees are expected to maintain confidentiality, and strict data security measures are in place. An annual budget of $1 million is allocated for cybersecurity training and awareness programs in the U.S., while budgets for Germany and Japan 

{'input': 'How much is the budget for Japan different than United States?',
 'output': "The budget for employee benefits in Japan is ¥50 million, while in the United States, it is $1 million. The leave-related expenses in Japan amount to ¥100 million, compared to $500,000 in the United States. \n\nTo provide a clearer comparison, I would need to convert yen to dollars, but the exact conversion rate varies. As of the last known rates, ¥50 million is approximately $460,000 and ¥100 million is approximately $920,000. Thus, Japan's budget for employee benefits is lower than the U.S.'s, while Japan's leave-related expenses are significantly higher. \n\nTherefore, the primary difference is that Japan spends more on leave-related expenses, while the U.S. spends more on employee benefits."}

In [ ]:
# dwa
tools = [tool_search, currency_conversion]

react_agent = create_react_agent(llm, tools=tools, prompt=prompt_react)
react_agent_executor = AgentExecutor(
    agent=react_agent, tools=tools, verbose=True, handle_parsing_errors=True
)

Te linie kodu tworzą system agenta z dwoma narzędziami. Rozbierzmy to:

`tools = [tool_search, currency_conversion]` tworzy listę zawierającą dwa narzędzia:
- narzędzie do wyszukiwania w dokumentach HR
- narzędzie do konwersji walut

`create_react_agent()` tworzy agenta ReAct z parametrami:
- `llm` - model językowy do wykorzystania
- `tools=tools` - lista dwóch dostępnych narzędzi
- `prompt=prompt_react` - wcześniej pobrany prompt określający sposób działania

`AgentExecutor()` tworzy wykonawcę dla tego agenta z parametrami:
- `agent=react_agent` - wcześniej utworzony agent
- `tools=tools` - ta sama lista narzędzi
- `verbose=True` - włącza szczegółowe logowanie działań
- `handle_parsing_errors=True` - włącza obsługę błędów parsowania

W porównaniu do poprzedniej wersji, agent ma teraz dostęp do dwóch narzędzi zamiast jednego - może zarówno przeszukiwać dokumenty HR, jak i dokonywać konwersji walut.

In [ ]:


query = "Is the budget for Japan different than United States?"
react_agent_executor.invoke({"input": query})



> Entering new AgentExecutor chain...
I need to look up the HR policy to find any information related to budget differences between Japan and the United States. 
Action: search_hr_policy 
Action Input: 'budget differences between Japan and United States' **Grievance and Disciplinary Procedures:**
Our grievance and disciplinary procedures are outlined on the company intranet. Termination conditions may include gross misconduct or repeated policy violations. In such cases, a disciplinary process will be followed, including a three-strike system, before termination. Employees leaving GlobalCorp should follow the exit process detailed in the employee handbook.

**Confidentiality and Data Security:**
Employees are expected to maintain confidentiality, and strict data security measures are in place. An annual budget of $1 million is allocated for cybersecurity training and awareness programs in the U.S., while budgets for Germany and Japan are €500,000 and ¥50 million, respectively.

**Com

{'input': 'Is the budget for Japan different than United States?',
 'output': 'Yes, the budget for Japan is different from that of the United States. Specifically, the annual budget for cybersecurity training and awareness programs in the U.S. is $1 million, while Japan has a budget of ¥50 million for employee benefits, and also an annual budget of ¥100 million for leave-related expenses.'}